## Lab3 - N-Puzzle

Solve efficiently a generic $n^2-1$ puzzle (also known as Gem Puzzle, Boss Puzzle, Mystic Square, etc.) using path-search algorithms.

In [1]:
from collections import namedtuple
from random import choice
import numpy as np
from queue import PriorityQueue

### Helper Functions

In [2]:
# Tuple to represent an action in the puzzle
action = namedtuple('Action', ['pos1', 'pos2'])

# Function to get available actions for a given state
def available_actions(state):
    n = state.shape[0]
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < n - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < n - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


# Function to perform an action on the current state
def do_action(state, action):
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


# Function to generate the solved puzzle
def generate_puzzle(PUZZLE_DIM):
    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return goal_state


# Function to shuffle the puzzle
def shuffle_puzzle(goal_state, RANDOMIZE_STEPS):
    start_state = goal_state.copy()
    for r in range(RANDOMIZE_STEPS):
        start_state = do_action(start_state, choice(available_actions(start_state)))
    return start_state

### Definition of the PuzzleState class

In [3]:
class PuzzleState:
    def __init__(self, state, g, path):
        self.state = state
        self.g = g
        self.path = path
        self.h = self.manhattan_distance()
        self.f = self.g + self.h

    # Calculate the Manhattan distance heuristic
    def manhattan_distance(self):
        distance = 0
        n = self.state.shape[0]
        for i in range(n):
            for j in range(n):
                if self.state[i, j] != 0:
                    x_goal, y_goal = divmod(self.state[i, j] - 1, n)
                    distance += abs(i - x_goal) + abs(j - y_goal)
        return distance

    # Check if two PuzzleState objects are equal (same state configuration)
    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

    # Generate a hash value for the PuzzleState object to be used in sets or dictionaries
    def __hash__(self):
        return hash(self.state.tobytes())

    # Compare PuzzleState objects based on the f value (used in priority queues)
    def __lt__(self, other):
        return self.f < other.f

    # Provide a string representation of the PuzzleState object for debugging or printing
    def __repr__(self):
        return f"\n{self.state}"

### Implement the A* algorithm

In [5]:
def a_star(start_state, goal_state):
    # Priority queue
    queue = PriorityQueue()
    start = PuzzleState(start_state, 0, [])
    goal_state = PuzzleState(goal_state, 0, [])
    # Insert the starting state
    queue.put(start)
    # States already visited
    visited = set()
    # Actions counter
    num_actions_evaluated = 0

    while not queue.empty():
        current_state = queue.get()

        # If we reach the goal
        if current_state == goal_state:
            return current_state.path, current_state.g, num_actions_evaluated

        # Avoid revisiting already explored states
        if current_state not in visited:
            visited.add(current_state)

            # Generate successors
            for action in available_actions(current_state.state):
                new_state = do_action(current_state.state, action)
                new_path = current_state.path + [action]
                new_g = current_state.g + 1
                new_state_obj = PuzzleState(new_state, new_g, new_path)
                queue.put(new_state_obj)
                num_actions_evaluated += 1

    # If no solution is found
    return None, -1, num_actions_evaluated

### Main code block to generate, shuffle, and solve the puzzle

In [6]:
def solve_puzzle(PUZZLE_DIM, RANDOMIZE_STEPS):
    # Generate the solved puzzle and shuffle it
    goal_state = generate_puzzle(PUZZLE_DIM)
    start_state = shuffle_puzzle(goal_state, RANDOMIZE_STEPS)
    print(f"Start state:\n{start_state}\n")

    # Solve the puzzle
    moves, num_solution_actions, num_actions_evaluated = a_star(start_state, goal_state)
    print(f"Solution found in {num_solution_actions} moves")
    print(f"Total number of actions evaluated: {num_actions_evaluated}")

### Solve the 8-puzzle (3x3)

In [ ]:
solve_puzzle(PUZZLE_DIM=3, RANDOMIZE_STEPS=100000)

Start state:
[[3 7 4]
 [6 0 8]
 [5 2 1]]

Solution found in 26 moves
Total number of actions evaluated: 3811


### Solve the 15-puzzle (4x4)

In [ ]:
solve_puzzle(PUZZLE_DIM=4, RANDOMIZE_STEPS=100000)

Start state:
[[ 0  9  6  8]
 [ 2 13 11  7]
 [10  4  5 12]
 [ 1 14  3 15]]

Solution found in 42 moves
Total number of actions evaluated: 1650528
